# 16.3 大语言模型部署与应用

## 16.3.1 Gradio简介
- Gradio是一个开源的 Python 库，它允许你快速地为机器学习模型创建可交互的网页界面。使用Gradio，研究人员和开发人员可以方便地展示他们的工作，使得非技术用户也能够轻松地尝试机器学习模型的功能，无需进行复杂的安装或设置。
- **核心特性：**
  - **易用性**：
    - Gradio 可以用几行代码就将机器学习模型变为交互式应用。它提供了各种输入和输出组件，如文本框、图像上传、滑块等。
  - **灵活性**：
    - 支持多种数据类型的输入和输出，包括文本、图像、音频和视频。
  - **兼容性**：
    - 可以与 TensorFlow、PyTorch、scikit-learn 等主流机器学习框架无缝集成。
  - **即时反馈**：
     - 用户可以实时看到他们输入数据的模型预测结果，这是一个强大的特性，尤其是在演示模型时。
  - **共享和部署**：
    - Gradio 提供了一个链接，通过这个链接其他用户可以访问你的模型，无需设置服务器或使用云服务。
  - **隐私保护**：
    - 所有数据都在用户的本地计算机上处理，不通过外部服务器。
  - **集成到科学生态系统**：
    - 可以集成到 Jupyter 笔记本或 Python 脚本中。
  - **定制化**：
    - 提供了样式和布局的自定义选项，以及用于更复杂交互的接口供了样式和布局的自定义选项，以及用于更复杂交互的接口。

- **高级用法：** Gradio 也支持更复杂的输入和输出类型，例如：
  - 输入/输出为图像的图像分类器界面。
  - 输入为文本，输出为音频的文本到语音系统界面。
  - 多输入多输出的界面，如同时接受图像和文本输入的模型。
- **定制和部署：**
  - 可以通过 CSS 定制 Gradio 界面的外观，也可以通过 `share` 参数在 Gradio 提供的服务器上临
  - 时部署你的界面，从而使其他人可以通过互联网访问它。
- **安装：**
```
pip install gradio
```

- **示例：简单应用**

In [1]:
# 一个最简单的示例
import gradio as gr

def greet(name,history):
    return "Hello, " + name  + "!"
gr.ChatInterface(fn=greet).launch()

## 16.3.1 利用Gradio部署开源大语言模型

In [1]:
from transformers import AutoTokenizer, AutoModel
import gradio as gr
# 模型保存路径，模型权重文件和配置文件保存在本程序代码上层目录huggingface中
model_path = "../huggingface/chatglm3-6b-32k"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# 加载模型的权重文件，然后将参数从float32转换成float16（half函数），最后将模型迁移到GPU内存（cuda函数）
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
# 将模型转换成评估状态
model = model.eval()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
response, history1 = model.chat(tokenizer, '你好', history=[])
def chat(name,history):
    print('history===========0',history)
    print('history1----------0',history1)
    response, history1 = model.chat(tokenizer, name, history=history1)
    print('history===========\n',history)
    print('history1----------\n',history1)
    return response

In [3]:
# 构造对话框，显示复制按钮
chatbot=gr.Chatbot(show_copy_button=True)
# 对话框的标签
chatbot.label="问答机器人"
# 构造输入框以及输入框提示
textbox = gr.Textbox(placeholder="在此处输入问题...", autofocus=True, scale=40)
# 构造对话界面
gr.ChatInterface(fn=chat, title="人工智能对话系统(测试版)",chatbot=chatbot,textbox =textbox,
                 submit_btn="提交",stop_btn="停止", retry_btn="🔄 重试",
                undo_btn="↩️ 撤销最后一个对话",clear_btn="🗑️ 清除历史对话").launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


history===========0
 []


Traceback (most recent call last):
  File "/home/user/.local/lib/python3.10/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/home/user/.local/lib/python3.10/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/home/user/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1554, in process_api
    result = await self.call_function(
  File "/home/user/.local/lib/python3.10/site-packages/gradio/blocks.py", line 1190, in call_function
    prediction = await fn(*processed_input)
  File "/home/user/.local/lib/python3.10/site-packages/gradio/utils.py", line 634, in async_wrapper
    response = await f(*args, **kwargs)
  File "/home/user/.local/lib/python3.10/site-packages/gradio/chat_interface.py", line 403, in _submit_fn
    response = await anyio.to_thread.run_sync(
  File "/home/user/.local/lib/python3.10/site-packages/anyio/to_thread.py", line 33, 

In [10]:
gr.Chatbot?

Init signature:
gr.Chatbot(
    value: 'list[list[str | tuple[str] | tuple[str | Path, str] | None]] | Callable | None' = None,
    color_map: 'dict[str, str] | None' = None,
    *,
    label: 'str | None' = None,
    every: 'float | None' = None,
    show_label: 'bool | None' = None,
    container: 'bool' = True,
    scale: 'int | None' = None,
    min_width: 'int' = 160,
    visible: 'bool' = True,
    elem_id: 'str | None' = None,
    elem_classes: 'list[str] | str | None' = None,
    height: 'int | None' = None,
    latex_delimiters: 'list[dict[str, str | bool]] | None' = None,
    rtl: 'bool' = False,
    show_share_button: 'bool | None' = None,
    show_copy_button: 'bool' = False,
    avatar_images: 'tuple[str | Path | None, str | Path | None] | None' = None,
    sanitize_html: 'bool' = True,
    render_markdown: 'bool' = True,
    bubble_full_width: 'bool' = True,
    line_breaks: 'bool' = True,
    layout: "Literal['panel', 'bubble'] | None" = None,
    **kwargs,
)
Docstring: 

In [11]:
model.chat?

Signature:
model.chat(
    tokenizer,
    query: str,
    history: List[Tuple[str, str]] = None,
    role: str = 'user',
    max_length: int = 32768,
    num_beams=1,
    do_sample=True,
    top_p=0.8,
    temperature=0.8,
    logits_processor=None,
    **kwargs,
)
Docstring: <no docstring>
File:      ~/.cache/huggingface/modules/transformers_modules/chatglm3-6b-32k/modeling_chatglm.py
Type:      method